<a href="https://colab.research.google.com/github/Approach-Analytics/Emotion-Classifier/blob/main/Dataprep_Fear_openwebtext2_July12%2C2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Zstandard seems to be a requirement for OpenWebText2
#Datasets is a download / library from HuggingFace

!pip install datasets
!pip install zstandard

In [2]:
import multiprocessing

# Determine the number of available CPU cores
num_cpus = multiprocessing.cpu_count()

# Output the number of available CPU cores
print("Number of CPU cores:", num_cpus)


Number of CPU cores: 40


In [3]:
#Import libraries...

import os
import re
import random
import pandas as pd
import pyarrow.parquet as pq
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import multiprocessing


In [4]:
#Setting display options

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set maximum number of rows and columns to display
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Work with the dataset

#Replace this dataset path with Google Cloud Storage...

In [6]:
from datasets import load_from_disk

dataset_path = "/content/drive/MyDrive/Sean/Data_OpenWebText2"

# Load the dataset from the file
dataset = load_from_disk(dataset_path)

# Now you can access the dataset
train_data = dataset['train']

In [7]:
#How big a folder?

def get_folder_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_folder_size(entry.path)
    return total

folder_path = "/content/drive/MyDrive/Sean/OpenWebText2"
size_in_bytes = get_folder_size(folder_path)

print(f'Size of the directory is {size_in_bytes / (1024*1024*1024)} GB')


FileNotFoundError: ignored

#Defining the Emotion Axis

These are the search terms...

In [8]:
#Defining the fear axis

emo_axis= {
    "dread" : ['panic','dread','horror','horrified','horrifying','terror','terrified','terrifying'],
    "fear":  ["fear", "fearful", "fright","frightening","afraid","frightful","frightfully","frightened",'scared','scary','scare'],
    "anxiety": ["anxious","anxiety",'angst','anxiousness'],
    "worry":["worry", "worried","worrying","worries"],
    "concern": ["concern", "concerning","concerned","concerns"],
    "calm": ["calm","peaceful","serene","serenity","untroubled", "content", "contented", "composed", "tranquil"]
    }



In [9]:
# Get the train split of the dataset
train_data = dataset['train']


In [10]:
train_data?


#More compute efficient method - Batching with Dataset Object...

I don't know if this works...

In [ ]:
import time
import re
import pandas as pd
from datasets import load_dataset

def create_patterns(search_dict):
    return {category: re.compile(fr"\b(?:{'|'.join(map(re.escape, words))})\b") for category, words in search_dict.items()}

def extract_context(match, text, window_size=50):
    start = max(0, match.start() - window_size)
    end = min(match.end() + window_size, len(text))
    return text[start:end]

def process_example(example, pattern_dict):
    text = example['text'].lower()
    text_list = []
    labl = []
    label2 = []

    for category, pattern in pattern_dict.items():
        matches = pattern.finditer(text)
        for match in matches:
            context = extract_context(match, text)
            if context not in text_list:
                text_list.append(context)
                labl.append(category)
                label2.append(category)

    return {"text_list": text_list, "label1": labl, "label2": label2}

# Set the search dictionary
emo_axis = {
    "dread": ['panic', 'dread', 'horror', 'horrified', 'horrifying', 'terror', 'terrified', 'terrifying'],
    "fear": ["fear", "fearful", "fright", "frightening", "afraid", "frightful", "frightfully", "frightened", 'scared', 'scary', 'scare'],
    "anxiety": ["anxious", "anxiety", 'angst', 'anxiousness'],
    "worry": ["worry", "worried", "worrying", "worries"],
    "concern": ["concern", "concerning", "concerned", "concerns"],
    "calm": ["calm", "peaceful", "serene", "serenity", "untroubled", "content", "contented", "composed", "tranquil"]
}

# Define the fraction of the dataset to use (e.g., 10%)
fraction = 0.05

# Start timer
start_time = time.time()


# Create the pattern dictionary
pattern_dict = create_patterns(emo_axis)

# Sample a subset of the dataset
sampled_dataset = train_data.shuffle().select(range(int(len(train_data) * fraction)))

# Process the sampled dataset using parallel processing
sampled_dataset = sampled_dataset.map(lambda example: process_example(example, pattern_dict), num_proc=40)

# Combine the results from each batch
text_list = []
labl = []
label2 = []

for result in sampled_dataset:
    text_list.extend(result["text_list"])
    labl.extend(result["label1"])
    label2.extend(result["label2"])

# Create a Pandas DataFrame from the lists
df = pd.DataFrame({'text': text_list, 'emotion1': labl, 'emotion2': label2})

# End timer
end_time = time.time()

# Print runtime
print("Runtime:", end_time - start_time, "seconds")


Map (num_proc=40):   0%|          | 0/855152 [00:00<?, ? examples/s]

#John Use this code...

John to refactor - create own notebook. That uses Google Cloud Storage / Buckets... Start with OpenWebText data extraction and Reddit....

In [ ]:
import re
import pandas as pd
from multiprocessing import Pool

def create_patterns(search_dict):
    # Combine all words in each category into a single regular expression
    return {category: re.compile(fr"\b(?:{'|'.join(map(re.escape, words))})\b") for category, words in search_dict.items()}

def process_example(example, pattern_dict):
    text = example['text'].lower()

    text_list = []
    labl = []
    label2 = []

    for category, pattern in pattern_dict.items():
        if pattern.search(text):
            text_list.append(text)
            labl.append(category)
            label2.append(category)

    return text_list, labl, label2

# The rest of your code...

pattern_dict = create_patterns(search_dict)

fraction = 0.05
sampled_dataset = train_data.shuffle().select(range(int(len(train_data) * fraction)))

#This will need to change given the number of CPU's available...

with Pool(40) as pool:    # Use the number of CPU cores available
    results = pool.starmap(process_example, [(example, pattern_dict) for example in sampled_dataset])

text_list = []
labl = []
label2 = []

for result in results:
    text_list.extend(result[0])
    labl.extend(result[1])
    label2.extend(result[2])

df = pd.DataFrame({'text': text_list, 'emotion1': labl, 'emotion2': label2})


In [ ]:
df.shape

In [ ]:
df.head(25)